In [2]:
import json
import shutil
import timeit
from collections import defaultdict
import traceback

from cognitive.auto_task.arguments import get_args
from cognitive import task_generator as tg
from cognitive import constants as const
from cognitive import stim_generator as sg
from cognitive import info_generator as ig
from cognitive.auto_task.auto_task_util import *

from api2nn import *

import unittest

from test.task_generator_test import TaskGeneratorTest

import numpy as np
import random
import networkx as nx
from tqdm import tqdm
from PIL import Image
import os
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout

from typing import Tuple, Union


In [ ]:
def generate_temporal_example(whens, first_shareable=None, temporal_switch=temporal_switch):
    # max_memory


    task = CompareLocTemporal( whens, first_shareable)
    assert isinstance(task, tg.TemporalTask)

    # To get maximum memory duration, we need to specify the following average
    # memory value
    avg_mem = round(max_memory / 3.0 + 0.01, 2)

    objset = task.generate_objset(n_distractor=0,average_memory_span=avg_mem, temporal_switch=temporal_switch)
   

    # Getting targets can remove some objects from objset.
    # Create example fields after this call.
    frame_info = ig.FrameInfo(task, objset)
    compo_info = ig.TaskInfoCompo(task, frame_info)
    return (compo_info, task)

In [ ]:
def generate_compo_temporal_example(n_tasks=1,
                                    whens=None, temporal_switch=temporal_switch):
    '''

    :param first_shareable:
    :param whens:
    :param families:
    :param max_memory:
    :param max_distractors:
    :param n_tasks:
    :return: combined TaskInfo Compo
    '''
    if n_tasks == 1:
        return generate_temporal_example(whens=whens, first_shareable=1, temporal_switch=temporal_switch)
    else:
        compo_tasks = [generate_temporal_example(whens=whens, first_shareable=1, temporal_switch=temporal_switch)
                    for _ in range(n_tasks)]
        # temporal combination
        cur_task = compo_tasks[0]
        for task in compo_tasks[1:]:
            cur_task.merge(task)
        return cur_task

In [ ]:
task_info, task = generate_compo_temporal_example(n_tasks=1, whens=[f'last{nback}', 'last0'], temporal_switch=temporal_switch)